In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [3]:
project_dir = "."
games = pd.read_csv(f'{project_dir}/games.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
players = pd.read_csv(f'{project_dir}/players.csv')
tracking_df = pd.read_csv(f'{project_dir}/tracking_week_2.csv')
player_play = pd.read_csv(f'{project_dir}/player_play.csv')

In [4]:
rushed_passer = player_play[['gameId', 'playId', 'nflId', 'wasInitialPassRusher']]

In [5]:
rushed_passer = rushed_passer[~rushed_passer['wasInitialPassRusher'].isna()]

In [6]:
line_set = tracking_df[tracking_df['event'] == 'line_set'][['gameId', 'playId', 'nflId', 'frameId']]

after_line_set = tracking_df.merge(line_set, on=['gameId', 'playId', 'nflId'], suffixes=["", "_lineset"])

after_line_set = after_line_set[(after_line_set['frameId'] >= after_line_set["frameId_lineset"]) & (after_line_set['frameType'] == 'BEFORE_SNAP')]


after_line_set = after_line_set.drop(['frameId_lineset'], axis=1)

defensive_motion = pd.merge(after_line_set, plays[['gameId', 'playId', 'defensiveTeam']], on = ['gameId', 'playId'])

#defensive_motion = pd.merge(defensive_motion, players[['nflId', 'displayName', 'position']], on = 'nflId', how = 'left')

defensive_motion = pd.merge(defensive_motion, rushed_passer, on = ['gameId', 'playId', 'nflId'], how = 'left')

defensive_motion = defensive_motion[defensive_motion['club'] == defensive_motion['defensiveTeam']]

In [7]:
defensive_motion = defensive_motion[defensive_motion['wasInitialPassRusher'] == 1]

In [8]:
defensive_motion = defensive_motion.groupby(['gameId', 'playId', 'nflId']).agg({'x':['first', 'last'], 'y':['first', 'last']}).reset_index()

In [9]:
defensive_motion.columns = [''.join(col).strip() for col in defensive_motion.columns.values]

In [10]:
defense_only_motion = defensive_motion[(np.abs(defensive_motion['ylast'] - defensive_motion['yfirst']) >= 2.5)]

In [11]:
defense_only_motion

,gameId,playId,nflId,xfirst,xlast,yfirst,ylast
97,2022091500,1191,43326.0,75.58,76.63,28.76,31.84
158,2022091500,1959,52546.0,71.20,70.88,21.41,24.29
215,2022091500,2625,52471.0,23.12,20.43,27.71,22.29
216,2022091500,2625,52546.0,23.81,20.30,32.58,35.79
232,2022091500,2731,43292.0,44.01,45.92,20.08,15.39
...,...,...,...,...,...,...,...
4169,2022091900,189,46457.0,51.35,53.00,28.47,31.17
4445,2022091901,454,48516.0,75.66,76.52,23.06,26.03
4450,2022091901,631,42431.0,32.46,31.12,20.98,18.22
4583,2022091901,2268,35452.0,67.25,67.43,32.31,23.67


In [12]:
defense_only_motion[['gameId', 'playId']].drop_duplicates()

,gameId,playId
97,2022091500,1191
158,2022091500,1959
215,2022091500,2625
232,2022091500,2731
374,2022091800,141
...,...,...
4130,2022091812,2969
4169,2022091900,189
4445,2022091901,454
4450,2022091901,631


In [13]:
tracking_df[['gameId', 'playId']].drop_duplicates()

,gameId,playId
0,2022091901,64
4301,2022091901,112
7291,2022091901,133
9108,2022091901,154
11799,2022091901,213
...,...,...
6692540,2022091500,3939
6694633,2022091500,3966
6698428,2022091500,3990
6700613,2022091500,4055


In [41]:
77/1952

0.03944672131147541

In [ ]:
#defensive_motion['distance'] = np.sqrt((defensive_motion['xlast'] - defensive_motion['xfirst']) ** 2 + (defensive_motion['ylast'] - defensive_motion['yfirst']) ** 2)
#defense_only_motion = motion[(motion['distance'] > 7.5) & ~(motion['motionSinceLineset'])]

#defense_only_motion = defensive_motion[(defensive_motion['s'] > 0.62) & (defensive_motion['dis'] > 1.2)]
#offense_motion = player_play.groupby(['gameId', 'playId'])[['motionSinceLineset']].max().reset_index()
#motion = pd.merge(defensive_motion, offense_motion, on = ['gameId', 'playId'])
#motion['motionSinceLineset'] = motion['motionSinceLineset'].fillna(False)

#defensive_motion = defensive_motion.groupby(['gameId', 'playId', 'nflId']).agg({'s':'max', 'dis':'sum'}).reset_index()
#defensive_motion['vert_move'] = defensive_motion['last'] - defensive_motion['first']